In [2]:
from mosaicml import describe_model

In [3]:
singapore_model = describe_model("b1504902-f222-479b-8db5-209e81dda8a6")

In [4]:
singapore_model["versions"][0]["deployments"][0]["deployment_info"]

{'configmap_name': 'jwt-dp-94c5bb65-2591-4732-af4e-5b28c47e4617',
 'cpu_utilization': '70',
 'deployment_name': 'dp-94c5bb65-2591-4732-af4e-5b28c47e4617',
 'deployment_type': 'Default',
 'host_name': 'livmosaic.lntinfotech.com/singaporemcmodelv3/b1504902-f222-479b-8db5-209e81dda8a6',
 'hpa_name': 'hpa-94c5bb65-2591-4732-af4e-5b28c47e4617',
 'ingress_name': 'ing-94c5bb65-2591-4732-af4e-5b28c47e4617',
 'init_configmap_name': 'script-dp-94c5bb65-2591-4732-af4e-5b28c47e4617',
 'labels': {},
 'service_name': 'svc-94c5bb65-2591-4732-af4e-5b28c47e4617',
 'strategy_deployment_id': '',
 'strategy_version_id': 'efc3264f-cd00-4011-ab7c-a32c5c30932c'}

In [38]:
import pandas as pds
def upsert_record(record):
    first_run = False
    try:
        df = pds.read_csv("/Users/nikhilbankar/docs/Work/Mosaic_AI/clients/MidCorpModels/output.csv")
    except:
        first_run = True
        
    if not first_run:
        try:
            index = df[(df["SCORECARD_ID"] == record["SCORECARD_ID"]) & (df["CoR"] == record["CoR"]) & (df["Scorecard_Segment_Type"] == record["Scorecard_Segment_Type"])].index[0]
            run_id = df.iloc[index]["run_id"]
            record["run_id"] = run_id + 1
            new_df = pds.DataFrame(record, index=[index])
            df.update(new_df)
        except:
            SCORECARD_ID = record["SCORECARD_ID"]
            CoR = record["CoR"]
            Scorecard_Segment_Type = record["Scorecard_Segment_Type"]
            Best_Possible_CRG = record["Best_Possible_CRG"]
            PD_grade_mapping = record["PD_grade_mapping"]
            pd = record["pd"]
            Final_Score = record["Final_Score"]
            Fin_Score = record["Fin_Score"]
            NonFin_Score = record["NonFin_Score"]
            Status = record["Status"]
            df = df.append(pds.DataFrame({'SCORECARD_ID': [SCORECARD_ID],
                                         "CoR":[CoR],
                                         "Scorecard_Segment_Type":[Scorecard_Segment_Type],
                                          "run_id":1,
                                         "Best_Possible_CRG":[Best_Possible_CRG],
                                         "PD_grade_mapping":[PD_grade_mapping],
                                         "pd":[pd],
                                         "Final_Score":[Final_Score],
                                         "Fin_Score":[Fin_Score],
                                         "NonFin_Score":[NonFin_Score],
                                          "Status" : Status
                                        }), 
                           ignore_index = True)
            
    else:
        SCORECARD_ID = record["SCORECARD_ID"]
        CoR = record["CoR"]
        Scorecard_Segment_Type = record["Scorecard_Segment_Type"]
        Best_Possible_CRG = record["Best_Possible_CRG"]
        PD_grade_mapping = record["PD_grade_mapping"]
        pd = record["pd"]
        Final_Score = record["Final_Score"]
        Fin_Score = record["Fin_Score"]
        NonFin_Score = record["NonFin_Score"]
        Status = record["Status"]
            
        df= pds.DataFrame({'SCORECARD_ID': [SCORECARD_ID],
                                         "CoR":[CoR],
                                         "Scorecard_Segment_Type":[Scorecard_Segment_Type],
                                        "run_id":1,
                                         "Best_Possible_CRG":[Best_Possible_CRG],
                                         "PD_grade_mapping":[PD_grade_mapping],
                                         "pd":[pd],
                                         "Final_Score":[Final_Score],
                                         "Fin_Score":[Fin_Score],
                                         "NonFin_Score":[NonFin_Score],
                                        "Status" : Status
                                        })
            
    
    df.to_csv("/data/output.csv",index = False)
    

In [2]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours

import json
class mydict(dict):
    def __str__(self):
        return json.dumps(self)
       
@scoring_func
def score(model, request):
    import requests
    import json
    payload = request.json
    payload = payload["payload"]
   
   
    try:
        input_dict = payload[0]

        financialData = payload[1]["financialData"]

        NonFinancialData = payload[2]["NonFinancialData"]

        for i in financialData:
            input_dict[i["factor"]] = i["factorValue"]
            input_dict[i["factorNumerator"]] = i["factorNumeratorValue"]
            input_dict[i["factorDenominator"]] = i["factorDenominatorValue"]


        for j in NonFinancialData:
            input_dict[j["factor"]] = j["factorValue"]
              
    except:
        return "error"
          
   
    if input_dict["cor"] == "SG":
       
        request_url = "http://svc-94c5bb65-2591-4732-af4e-5b28c47e4617:5001/singaporemcmodelv3/b1504902-f222-479b-8db5-209e81dda8a6/score"

        response = requests.post(request_url,json = input_dict).json()
        out= json.loads(response['data'].replace("\n",""))
       
    elif input_dict["cor"] == "UAE":
       
        request_url = "http://svc-3a6bc47a-250d-4002-9266-8f5a4990b01a:5001/uaemcmodelwithnumdem/9ddb1127-60f7-460f-829c-253d491c72ce/score"


        response = requests.post(request_url,json = input_dict).json()
        out= json.loads(response['data'].replace("\n",""))
        
    else:
        out = {"model_type": "No Country passed"}
        
    # csv file logic
    try:
        upsert_record(out)
    except:
        pass
    return out

ModuleNotFoundError: No module named 'mosaicml'

In [8]:
model = 10
register_model(model,
               score,
               "IRB_controller",
               "IRB_controller",
               MLModelFlavours.sklearn,
               input_type="json",
               init_script="mkdir abc \\n cd abc \\n touch abc.txt"
              )

Exception: Unable to register model. Please try after some time

In [37]:
#Status = "compute"
#
out = {'SCORECARD_ID': 'e123',
 'CoR': 'SG',
 'Scorecard_Segment_Type': 'MC',
 'Best_Possible_CRG': '7a',
 'PD_grade_mapping': '3a',
 'pd': 0.01,
 'Final_Score': 19,
 'Fin_Score': 21,
 'NonFin_Score': 0,
 'run_id': 3.0,
"Status":"compute"}

upsert_record(out)

In [1]:
# testing the code

import requests
import json
payload = {"Benchmk": "2 - 2nd quartile",
 "BuzAge": 10,
 "Cash_flow_ratio": 2.7878864,
 "Cash_flow_ratio_denominator_new": 111858089,
 "Cash_flow_ratio_numerator_new": 311847645.2,
 "CoR": "SC",
 "Debt_To_EBITDA": 0,
 "EBITDA_new": 12363981,
 "GR_PR_LS_FRM_TRD_new": 27183655,
 "Grs_Pr_Mar": 0.14404555,
 "IndProsp": "2 - Stable",
 "Leverage": 0.7451909,
 "Leverage_denominator_new": 170311694,
 "Leverage_numerator_new": 126914717,
 "Parental_Support": "Yes",
 "SCORECARD_ID": "a123",
 "SLS_REVENUES_new": 188715689,
 "Sales_Turnover": 1000,
 "Scorecard_Segment_Type": "MC",
 "Total_Debt_new": 0}

request_url = "http://svc-94c5bb65-2591-4732-af4e-5b28c47e4617:5001/singaporemcmodelv3/b1504902-f222-479b-8db5-209e81dda8a6/score"

#response = requests.post(request_url,data = payload).json()


requests.post(request_url,json = payload).json()

ConnectionError: HTTPConnectionPool(host='svc-94c5bb65-2591-4732-af4e-5b28c47e4617', port=5001): Max retries exceeded with url: /singaporemcmodelv3/b1504902-f222-479b-8db5-209e81dda8a6/score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feeb5f94460>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))